In [1]:
!pip install requests beautifulsoup4

In [2]:
import pandas as pd
drug_data = pd.read_csv("/content/Drug_Data.csv")

In [3]:
drug_data.head()

,drugName,Prescribed_for,Drug_Review,User_Rating,Date,Count_of_Reviews
0,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10,28-Feb-12,22
1,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8,17-May-09,17
2,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9,29-Sep-17,3
3,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9,05-Mar-17,35
4,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9,22-Oct-15,4


In [4]:
disease = drug_data.Prescribed_for.unique()

#Web_Scrape from seattlechildrens.org: https://www.seattlechildrens.org/conditions/a-z/

In [24]:
import requests
from bs4 import BeautifulSoup

base_url = 'https://www.seattlechildrens.org'

# List of diseases
diseases = disease[:236]

symptoms = {}

# Get list of disease links from A-Z page
url = base_url + '/conditions/a-z/'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

disease_links = []
for link in soup.find_all('a'):
  if link.text in diseases:
    disease_links.append(link.get('href'))

print(disease_links)

['/conditions/a-z/acne/', '/conditions/anxiety/', '/conditions/back-pain/', '/conditions/a-z/constipation/', '/conditions/a-z/cough/', '/conditions/a-z/depression/', '/conditions/depression/', '/conditions/a-z/diarrhea/', '/conditions/a-z/eczema/', '/conditions/epilepsy/', '/conditions/a-z/headache/', '/conditions/heart-failure/', '/conditions/a-z/motion-sickness/', '/conditions/pulmonary-hypertension/', '/conditions/supraventricular-tachycardia/', '/conditions/ulcerative-colitis/', '/conditions/a-z/warts/']


In [27]:
import re

def get_symptoms(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    headings = soup.find_all(re.compile('h[1-6]'), text=re.compile('.*Symptom.*'))
    for h in headings:
        ul = h.find_next('ul')
        if ul:
            symptoms = [li.text for li in ul.find_all('li')]
            return symptoms

    return None

# Extract symptoms from each disease page
for link in disease_links:

  page = requests.get(base_url + link)
  soup = BeautifulSoup(page.content, 'html.parser')

  disease = soup.find('h1').text


  symptoms[disease] = get_symptoms(base_url + link)

# Remove diseases with None values
to_remove = []
for disease, symptom_list in symptoms.items():
  if symptom_list is None:
    to_remove.append(disease)

for disease in to_remove:
  del symptoms[disease]

print(symptoms)

<ipython-input-27-486707e8742f>:7: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  headings = soup.find_all(re.compile('h[1-6]'), text=re.compile('.*Symptom.*'))


{'Acne': ['Whiteheads (pimples) are plugged oil glands that are closed.', 'Blackheads are plugged oil glands that are open. Reason: The oil turns black when it is exposed to air.', 'Whiteheads and blackheads are also called "zits."', 'Red bumps are from blocked oil glands that have leaked oil. This causes irritation in the skin around them. Larger red bumps can be quite painful.', 'Acne mainly appears on your face, neck, and shoulders'], 'Depression': ['Sadness or feeling irritable', 'Loss of interest or pleasure in usual activities', 'Eating too much or too little', 'Weight changes', 'Sleeping too much or too little', 'Feeling tired a lot', 'Feeling guilty', 'Trouble thinking or paying attention', 'Suicidal thoughts or behavior'], 'Eczema': ["The main symptom is itching. If it doesn't itch, it's not eczema.", 'With flare-ups (itching attacks), the rash becomes red or even raw and weepy.', 'Onset: Average onset at 3 months old. Range: 1-6 months old. Usually begins by 2 years old.', 'L

In [28]:
import pandas as pd

processed_symptoms = {}
for disease, symptom_list in symptoms.items():
    for symptom in symptom_list:
        processed_symptoms[disease] = symptom

df = pd.DataFrame.from_dict(processed_symptoms, orient='index')
df = df.reset_index().rename(columns={'index':'Disease', 0:'Symptoms'})

print(df)

                        Disease  \
0                          Acne   
1                    Depression   
2                        Eczema   
3               Motion Sickness   
4        Pulmonary Hypertension   
5  Supraventricular Tachycardia   
6            Ulcerative Colitis   
7                         Warts   

                                            Symptoms  
0  Acne mainly appears on your face, neck, and sh...  
1                      Suicidal thoughts or behavior  
2  The rash is usually the same on both sides of ...  
3  After age 12, the main symptom is nausea (feel...  
4                             Swelling in the ankles  
5  An event recorder can be turned on by your chi...  
6                                Rarely, blood clots  
7  Not painful unless located on the sole of the ...  


In [29]:
df

,Disease,Symptoms
0,Acne,"Acne mainly appears on your face, neck, and sh..."
1,Depression,Suicidal thoughts or behavior
2,Eczema,The rash is usually the same on both sides of ...
3,Motion Sickness,"After age 12, the main symptom is nausea (feel..."
4,Pulmonary Hypertension,Swelling in the ankles
5,Supraventricular Tachycardia,An event recorder can be turned on by your chi...
6,Ulcerative Colitis,"Rarely, blood clots"
7,Warts,Not painful unless located on the sole of the ...


## Web_Scrape from wikipedia:

In [6]:
import re
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")



def search_wikipedia_for_url(search_term):

    endpoint = f"https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "list": "search",
        "srsearch": search_term
    }

    response = requests.get(endpoint, params=params)

    if response.status_code == 200:
        data = response.json()
        search_results = data.get("query", {}).get("search", [])
        if search_results:
            return search_results




def web_scrape(disease_list):

    """
    disease_list: input list takes list of disease to search and collect the symptoms from wikipedia
    diseases_with_symptoms: Returns dictionary containing diseases and its symptoms
    """
    diseases_with_symptoms = {}
    for disease in disease_list:
        url_link = search_wikipedia_for_url(disease)

        if url_link:
            for url in url_link:

                page_title = url['title']

                page_url = f"https://en.wikipedia.org/wiki/{page_title.replace(' ', '_')}"

                wiki = requests.get(page_url, verify=False)
                soup = BeautifulSoup(wiki.content, 'html5lib')
                info_table = soup.find("table", {"class": "infobox"})
                if info_table:
                    for row in info_table.find_all("tr"):
                        header = row.find("th", {"scope": "row"})
                        if header and header.get_text() == "Symptoms":
                            symptom = row.find("td")
                            if symptom:
                                symptom = ' '.join(re.sub(r'<.*?>|\[.*?\]', ' ', str(symptom)).split())
                                diseases_with_symptoms[disease] = symptom

                if disease in diseases_with_symptoms.keys():
                    break

    return diseases_with_symptoms
disease_list = disease[:236]
disease_with_symptoms = web_scrape(disease_list)

In [8]:
prescribed_for, symptoms = [], []
for key, value in disease_with_symptoms.items():
    prescribed_for.append(key)
    symptoms.append(value)

In [9]:
symptoms_df = pd.DataFrame({'Prescribed_for': prescribed_for, 'Symptoms': symptoms}, index=range(1, len(disease_with_symptoms) + 1))


In [10]:
symptoms_df

,Prescribed_for,Symptoms
1,Depression,"Low mood, aversion to activity, loss of intere..."
2,"Crohn's Disease, Maintenance","Abdominal pain , diarrhea (may be bloody), fev..."
3,Urinary Tract Infection,"Pain with urination , frequent urination, feel..."
4,Keratosis,Firmly attached white patch on a mucous membra...
5,Migraine Prevention,"Headaches , nausea , sensitivity to light , so..."
...,...,...
181,Urinary Tract Stones,"Severe pain in the lower back or abdomen, bloo..."
182,Left Ventricular Dysfunction,"Shortness of breath , exhaustion , swollen legs"
183,"Anemia, Chemotherapy Induced","pale skin , fatigue , fast heart rate , rash ,..."
184,Traveler's Diarrhea,"Unformed stool while traveling, fever, abdomin..."
